<a href="https://colab.research.google.com/github/gnodking7/PINN-California-Delta/blob/main/PINN_ANN_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to find a set of optimal hyperparameters for ANN (fully-connected) and a set of optimal hyperparameters for PINN (Physics-informed neural network).

Both NN models estimate the salinity at locations Martinez, Port Chicago, Chipps Island stations in California-Delta estuary using the outflow data at these three locations as inputs.

The main difference between ANN model and PINN model is illustrated by the following NN structure:



1.   ANN

![](https://drive.google.com/uc?id=1pPT0dd_v91PRnfj7lw8aLIzWZu0T9Rcu)

*   The input of ANN is the outflow vector $\vec{Q}_n$ corresponding to day $n$.
*   ANN is trained by minimizing the mean squared error $$\sum_n\|\hat{S}_n-S_n\|^2$$ where $\hat{S}_n$ is the output of ANN and $S_n$ is the true salinity value.

2.   PINN

![](https://drive.google.com/uc?id=1Z77TmoPzrmUN3SdADFWhaD22KXC9LY5f)

*   The input of PINN is the outflow vector $\vec{Q}_n$ **and** location $x_n$ and time $t_n$.
*   PINN is trained by minimizing the mean squared error **and** PDE(1D Advection Dispersion eq) loss $$\sum_n\|\hat{S}_n-S_n\|^2 +\sum_n\bigg\|A\frac{\partial \hat{S}}{\partial t}\Bigr|_{(x_n,t_n)}-\vec{Q}_{n,1}\frac{\partial \hat{S}}{\partial x}\Bigr|_{(x_n,t_n)}-KA\frac{\partial^2 \hat{S}}{\partial x^2}\Bigr|_{(x_n,t_n)}\bigg\|^2$$ where $\vec{Q}_{n,1}$ is the first component of the outflow vector $\vec{Q}_n$.

# Preliminary Setup

## Install packages


In [ ]:
!pip install -q -U keras-tuner
!pip install deepxde

     |████████████████████████████████| 135 kB 30.8 MB/s 
     |████████████████████████████████| 1.6 MB 45.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 144 kB 17.9 MB/s 
     |████████████████████████████████| 100 kB 11.4 MB/s 


In [ ]:
import keras_tuner
from tensorflow import keras
import tensorflow as tf
import deepxde as dde

DeepXDE backend not selected or invalid. Use tensorflow.compat.v1.
Using backend: tensorflow.compat.v1

Instructions for updating:
non-resource variables are not supported in the long term


Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Enable just-in-time compilation with XLA.



In [ ]:
import autograd.numpy as np
import matplotlib.pylab as pl
import pandas as pd
import sys

## Read Data

### Read outflow and salinity data

In [ ]:
sys.path.append('/content/drive/My Drive/MAC/Python')

In [ ]:
'''
The data is in an Excel file.
Daily DSM2 outflow and salinity at locations:
  Martinez  Port Chicago  Chipps Island
from Apr 1 1990 to Dec 28 2017
'''
df = pd.read_excel("/content/drive/My Drive/MAC/Python/PortChicago202210.xlsx", header=None)
All = np.asarray(df)
All = All[7:, :]  # Discard headers

for i in range(len(All)):
    All[i, 1] = str(All[i, 1])  # Change dates to str

import numpy

In [ ]:
def data_pre(year, daily, window, window_len):
    """
    Returns preprocessed outflow data for a given 'year' at locations
    Martinez, Port Chicago, Chipps Island such that
    to each day corresponds outflow data of length 'daily' + 'window'

    The preprocessed outflow data vector is created such that
    first 'daily' number of outflow (including current day) is included
    and prior 'window_len' many outflows are averaged 'window' many times

    PARAMETERS
    ----------
    year :        string or array of strings
                  Outflow year(s)
    daily :       int
                  number of outflow to include 'as-is'
    window :      int
                  number of average windows in return outflow
    window_len :  int
                  length of each window
    RETURNS
    -------
    M_OUT :       ndarray, shape (year length, daily+window)
                  Preprocessed outflow at Martinez
    P_OUT :       ndarray, shape (year length, daily+window)
                  Preprocessed outflow at Port Chicago
    C_OUT :       ndarray, shape (year length, daily+window)
                  Preprocessed outflow at Chipps Island
    """

    if type(year) == str:
        indices = numpy.flatnonzero(numpy.core.defchararray.find(list(All[:, 1]), year)!=-1)
    else:
        indices = []
        for yr in year:
            indices.extend(numpy.flatnonzero(numpy.core.defchararray.find(list(All[:, 1]), yr)!=-1))
    total = daily + window * window_len
    OUT_d = All[indices[0] - (total - 1):indices[-1] + 1, 2:5] # Range of outflow to extract outflow vector from
    OUT_d = np.flip(OUT_d, 1) # Locations are in reverse order, so flip data
    OUT_d = (OUT_d - np.min(OUT_d)) / (np.max(OUT_d) - np.min(OUT_d)) # Normalize

    ## Preprocessed outflow data for Martinez
    M_OUT = []
    cur = total - 1
    for i in range(len(indices)):
        vec = OUT_d[cur-(daily-1):cur+1, 0]
        vec = np.flip(vec)
        ind = cur - (daily - 1)
        for j in range(window):
            avg = np.mean(OUT_d[ind-window_len:ind, 0])
            vec = np.concatenate((vec, np.array([avg])))
            ind -= window_len
        M_OUT.append(vec)
        cur += 1
    M_OUT = np.array(M_OUT)

    ## Preprocessed outflow data for Port Chicago
    P_OUT = []
    cur = total - 1
    for i in range(len(indices)):
        vec = OUT_d[cur-(daily-1):cur+1, 1]
        vec = np.flip(vec)
        ind = cur - (daily - 1)
        for j in range(window):
            avg = np.mean(OUT_d[ind-window_len:ind, 1])
            vec = np.concatenate((vec, np.array([avg])))
            ind -= window_len
        P_OUT.append(vec)
        cur += 1
    P_OUT = np.array(P_OUT)

    ## Preprocessed outflow data for Chipps Island
    C_OUT = []
    cur = total - 1
    for i in range(len(indices)):
        vec = OUT_d[cur-(daily-1):cur+1, 2]
        vec = np.flip(vec)
        ind = cur - (daily - 1)
        for j in range(window):
            avg = np.mean(OUT_d[ind-window_len:ind, 2])
            vec = np.concatenate((vec, np.array([avg])))
            ind -= window_len
        C_OUT.append(vec)
        cur += 1
    C_OUT = np.array(C_OUT)

    return M_OUT, P_OUT, C_OUT

### Preprocess to create outflow vectors

In [ ]:
years = ['1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']

indices = []
for yr in years:
    indices.extend(numpy.flatnonzero(numpy.core.defchararray.find(list(All[:, 1]), yr)!=-1))
indices = np.array(indices)

EC_d = All[indices, 5:] # Salinity at three locations
EC_min = np.min(EC_d)
EC_max = np.max(EC_d)

'''
Create outflow data vectors at each location
To each day corresponds a 18-dim vector created from antecedent 118 outflow data:
  current day + previous 7 days + previous 110 days averaged into 10-dim values with window length 11
'''
M_OUT, P_OUT, C_OUT = data_pre(years, 8, 10, 11)

### Split to train and validation

In [ ]:
tr_years = ['1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
# tr_years = ['1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011']
tst_years = ['1991','1992','1993','1994','1995','1996']
# tst_years = ['2012','2013','2014','2015','2016','2017']

tr_indices = []
for yr in tr_years:
    tr_indices.extend(numpy.flatnonzero(numpy.core.defchararray.find(list(All[:, 1]), yr)!=-1))
tr_indices = np.array(tr_indices)

tst_indices = []
for yr in tst_years:
    tst_indices.extend(numpy.flatnonzero(numpy.core.defchararray.find(list(All[:, 1]), yr)!=-1))
tst_indices = np.array(tst_indices)

In [ ]:
## Training data
tr_EC_d = All[tr_indices, 5:]
tr_EC_d = np.flip(tr_EC_d, 1)
tr_EC_d = (tr_EC_d - EC_min) / (EC_max - EC_min) # Normalize

Q = []  # Inputs for ANN
X = []  # Inputs for PINN
Svals = []  # Outputs (same for both ANN and PINN)

portion = 1 / (len(tr_indices) + len(tst_indices) # to split the time domain [0, 1] uniformly

for i in range(len(tr_EC_d)):
    cur_ind = tr_indices[i]
    for j in range(3):
        # Martinez
        if j == 0:
            Q.append(M_OUT[cur_ind-275, :])
            X.append(np.concatenate(([0, (cur_ind - 274) * portion], M_OUT[cur_ind-275, :])))
        # Port Chicago
        elif j == 1:
            Q.append(P_OUT[cur_ind-275, :])
            X.append(np.concatenate(([0.442, (cur_ind - 274) * portion], P_OUT[cur_ind-275, :])))
        # Chipps Island
        elif j == 2:
            Q.append(C_OUT[cur_ind-275, :])
            X.append(np.concatenate(([1, (cur_ind - 274) * portion], C_OUT[cur_ind-275, :])))
        Svals.append(tr_EC_d[i, j])

Q = np.array(Q).astype('float32')
X = np.array(X).astype('float32')
Svals = np.array(Svals).astype('float32').flatten()[:, None]

## Testing data
tst_EC_d = All[tst_indices, 5:]
tst_EC_d = np.flip(tst_EC_d, 1)
tst_EC_d = (tst_EC_d - EC_min) / (EC_max - EC_min) # Normalize

Q_tst = []  # Validation inputs for ANN
X_tst = []  # Validation inputs for PINN
Svals_tst = []  # Validation outputs

for i in range(len(tst_EC_d)):
    cur_ind = tst_indices[i]
    for j in range(3):
        # Martinez
        if j == 0:
            Q_tst.append(M_OUT[cur_ind-275, :])
            X_tst.append(np.concatenate(([0, (cur_ind - 274) * portion], M_OUT[cur_ind-275, :])))
        # Port Chicago
        elif j == 1:
            Q_tst.append(P_OUT[cur_ind-275, :])
            X_tst.append(np.concatenate(([0.442, (cur_ind - 274) * portion], P_OUT[cur_ind-275, :])))
        # Chipps Island
        elif j == 2:
            Q_tst.append(C_OUT[cur_ind-275, :])
            X_tst.append(np.concatenate(([1, (cur_ind - 274) * portion], C_OUT[cur_ind-275, :])))
        Svals_tst.append(tst_EC_d[i, j])

Q_tst = np.array(Q_tst).astype('float32')
X_tst = np.array(X_tst).astype('float32')
Svals_tst = np.array(Svals_tst).astype('float32').flatten()[:, None]

# Keras Tuner

## ANN model

### Set hyperparameters search space

In [ ]:
'''
Neural Network structure:
    1. Number of layers is preset to be

                    Act 1                Act 2                Act 3
        Input Layer ----> Hidden Layer 1 ----> Hidden Layer 2 ----> Output Layer
    
    2. Possible number of neurons for Hidden Layer 1 is [4, 8, 12, ..., 32]
    3. Possible number of neurons for Hidden Layer 2 is [2, 4, 6, ..., 16]
    4. Possible activation function for all three activation functions is ["relu", "tanh", "elu", "sigmoid"]
'''

def build_model(hp):
    # Hyperparameters
    act_func_1=hp.Choice("activation_1", ["relu","tanh","elu","sigmoid"])
    act_func_2=hp.Choice("activation_2", ["relu","tanh","elu","sigmoid"])
    act_func_3=hp.Choice("activation_3", ["relu","tanh","elu","sigmoid"])
    hidden_units_1 = hp.Int("nhidden1", min_value=4, max_value=32, step=4)
    hidden_units_2 = hp.Int("nhidden2", min_value=2, max_value=16, step=2)
        
    model = keras.Sequential(
        [
            keras.layers.Input(shape=(18,)),
            keras.layers.Dense(hidden_units_1, activation=act_func_1),
            keras.layers.Dense(hidden_units_2, activation=act_func_2),
            keras.layers.Dense(1, activation=act_func_3)
        ])
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=0.001), loss="mse")
    return model

build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss", # Find a set of hyperparameters that minimizes the validation loss
    max_trials=50,  # Randomly search for 50 sets
    executions_per_trial=2, # For each set, run twice
    # overwrite=True,
    # directory='tuner',
    # project_name="ann_dsm2",
)

tuner.search_space_summary()

Search space summary
Default search space size: 5
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
activation_3 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
nhidden1 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': None}
nhidden2 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 16, 'step': 2, 'sampling': None}


### Search

In [ ]:
tuner.search(Q, Svals,
                epochs=3000,
                batch_size=len(Q),  # entire training for 1 epoch
                validation_data=(Q_tst, Svals_tst),
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        monitor="val_loss", patience=50, mode="min", restore_best_weights=True),
                    #tensorboard_cb
                ],
                 verbose=0
                )

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [ ]:
## 10 best sets of hyperparameters

tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation_1: relu
activation_2: tanh
activation_3: elu
nhidden1: 24
nhidden2: 10
Score: 0.0077597759664058685
Trial summary
Hyperparameters:
activation_1: relu
activation_2: relu
activation_3: elu
nhidden1: 8
nhidden2: 14
Score: 0.0077623603865504265
Trial summary
Hyperparameters:
activation_1: relu
activation_2: tanh
activation_3: elu
nhidden1: 20
nhidden2: 12
Score: 0.007827656110748649
Trial summary
Hyperparameters:
activation_1: relu
activation_2: elu
activation_3: tanh
nhidden1: 24
nhidden2: 14
Score: 0.007977251894772053
Trial summary
Hyperparameters:
activation_1: elu
activation_2: relu
activation_3: elu
nhidden1: 12
nhidden2: 12
Score: 0.008038962725549936
Trial summary
Hyperparameters:
activation_1: elu
activation_2: relu
activation_3: tanh
nhidden1: 24
nhidden2: 14
Score: 0.00811866857111454
Trial summary
Hyperparameters:
activation_1: relu
activation_2: elu
activation_3: tanh

## PINN model

### Set hyperparameters search space

In [ ]:
""" 
Custom loss function for PINN
    Mean squared error + PDE (1D Advection Dispersion) loss
"""
def PINN_loss(inp):
    def loss(y_true, y_pred):
        A = 0.002
        K = 133.959
        S = y_pred
        Q = inp[2:3]
        dS_x = dde.grad.jacobian(S, inp, i=0, j=0)
        dS_t = dde.grad.jacobian(S, inp, i=0, j=1)
        dS_xx = dde.grad.hessian(S, inp, i=0, j=0)
        mse = tf.keras.losses.MeanSquaredError()
        return mse(y_true, S) + ((A * dS_t - Q * dS_x - K * A * dS_xx) ** 2)
    return loss

In [ ]:
'''
Neural Network structure:
    1. Number of layers is preset to be

                    Act 1                Act 2                Act 3
        Input Layer ----> Hidden Layer 1 ----> Hidden Layer 2 ----> Output Layer
    
    2. Possible number of neurons for Hidden Layer 1 is [4, 8, 12, ..., 32]
    3. Possible number of neurons for Hidden Layer 2 is [2, 4, 6, ..., 16]
    4. Possible activation function for all three activation functions is ["relu", "tanh", "elu", "sigmoid"]
'''

def build_model(hp):
    # Hyperparameters
    act_func_1=hp.Choice("activation_1", ["relu","tanh","elu","sigmoid"])
    act_func_2=hp.Choice("activation_2", ["relu","tanh","elu","sigmoid"])
    act_func_3=hp.Choice("activation_3", ["relu","tanh","elu","sigmoid"])
    act_func_4=hp.Choice("activation_4", ["relu","tanh","elu","sigmoid"])
    hidden_units_1 = hp.Int("nhidden1", min_value=2, max_value=50, step=4)
    hidden_units_2 = hp.Int("nhidden2", min_value=2, max_value=50, step=4)
    hidden_units_3 = hp.Int("nhidden3", min_value=2, max_value=50, step=4)

    Inp = keras.layers.Input(shape=(20,))
    L1 = keras.layers.Dense(hidden_units_1, activation=act_func_1)(Inp)
    L2 = keras.layers.Dense(hidden_units_2, activation=act_func_2)(L1)
    L3 = keras.layers.Dense(hidden_units_3, activation=act_func_2)(L2)
    Out = keras.layers.Dense(1, activation=act_func_4)(L3)
    PINN_model = keras.Model(Inp, Out)
    PINN_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=PINN_loss(Inp))
    return PINN_model

build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss", # Find a set of hyperparameters that minimizes the validation loss
    max_trials=50,  # Randomly search for 50 sets
    executions_per_trial=2, # For each set, run twice
    # overwrite=True,
    # directory='tuner',
    # project_name="ann_dsm2",
)

tuner.search_space_summary()

Search space summary
Default search space size: 7
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
activation_3 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
activation_4 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'elu', 'sigmoid'], 'ordered': False}
nhidden1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 50, 'step': 4, 'sampling': None}
nhidden2 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 50, 'step': 4, 'sampling': None}
nhidden3 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 50, 'step': 4, 'sampling': None}


### Search

In [ ]:
tuner.search(X, Svals,
                epochs=3000,
                batch_size=len(X),  # entire training for 1 epoch
                validation_data=(X_tst, Svals_tst),
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        monitor="val_loss", patience=50, mode="min", restore_best_weights=True),
                    #tensorboard_cb
                ],
                verbose=0
                )

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [ ]:
## 10 best sets of hyperparameters

tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation_1: relu
activation_2: tanh
activation_3: tanh
activation_4: sigmoid
nhidden1: 10
nhidden2: 26
nhidden3: 14
Score: 0.005956381559371948
Trial summary
Hyperparameters:
activation_1: sigmoid
activation_2: relu
activation_3: sigmoid
activation_4: elu
nhidden1: 14
nhidden2: 30
nhidden3: 46
Score: 0.006114940391853452
Trial summary
Hyperparameters:
activation_1: sigmoid
activation_2: relu
activation_3: sigmoid
activation_4: elu
nhidden1: 14
nhidden2: 42
nhidden3: 26
Score: 0.00656454567797482
Trial summary
Hyperparameters:
activation_1: sigmoid
activation_2: relu
activation_3: sigmoid
activation_4: elu
nhidden1: 46
nhidden2: 6
nhidden3: 46
Score: 0.0067662259098142385
Trial summary
Hyperparameters:
activation_1: relu
activation_2: sigmoid
activation_3: relu
activation_4: relu
nhidden1: 6
nhidden2: 2
nhidden3: 30
Score: 0.006785990204662085
Trial summary
Hyperparameters:
activation_1